# 🚀 Text Generation Apps using Python (Completions Only)

This notebook showcases **14 text generation use cases** using OpenAI **Completions API**.  
Each section includes a description, the Python code, and expected output.

---

## 📑 Table of Contents
1. [Summarization](#-Use-Case-1-Summarization)  
2. [Sentiment Classification](#-Use-Case-2-Sentiment-Classification)  
3. [Multilingual Generation + Translation](#-Use-Case-3-Multilingual-Generation--Translation)  
4. [Semantic Interpretation of Idiom](#-Use-Case-4-Semantic-Interpretation-of-Idiom)  
5. [Factual Recall / Explanatory Response](#-Use-Case-5-Factual-Recall--Explanatory-Response)  
6. [Code Generation & Explanation](#-Use-Case-6-Code-Generation--Explanation)  
7. [Conversational Agent (FAQ Assistant)](#-Use-Case-7-Conversational-Agent-FAQ-Assistant)  
8. [Style Transfer / Tone Adaptation](#-Use-Case-8-Style-Transfer--Tone-Adaptation)  
9. [Data-to-Text Generation](#-Use-Case-9-Data-to-Text-Generation)  
10. [Creative Writing](#-Use-Case-10-Creative-Writing)  
11. [Question Generation](#-Use-Case-11-Question-Generation)  
12. [Entity Extraction with Explanation](#-Use-Case-12-Entity-Extraction-with-Explanation)  
13. [Paraphrasing / Rewriting](#-Use-Case-13-Paraphrasing--Rewriting)  
14. [Email / Document Drafting](#-Use-Case-14-Email--Document-Drafting)  

---


## Setup

Load environment variables (OPENAI_API_KEY) using python-dotenv.

Create a helper function get_completion(prompt) that calls the OpenAI Completions API.

In [ ]:
# Install required packages (skip if already installed)
# %pip install openai tiktoken python-dotenv

import os
import logging
from dotenv import load_dotenv
from openai import OpenAI
from typing import Optional

# -------------------------------------------------------------------
# Setup logging (INFO by default, DEBUG for detailed traces)
# -------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
# Load environment variables
# -------------------------------------------------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in environment or .env file")

# -------------------------------------------------------------------
# Initialize OpenAI client
# -------------------------------------------------------------------
client = OpenAI(api_key=OPENAI_API_KEY)

# -------------------------------------------------------------------
# Helper function for completions API
# -------------------------------------------------------------------


def get_completion(
    prompt: str,
    model: str = "gpt-3.5-turbo-instruct",
    max_tokens: int = 200,
    temperature: float = 0.7,
    log_response: bool = False
) -> Optional[str]:
    """
    Get a text completion from OpenAI API.

    Args:
        prompt (str): The input prompt.
        model (str): Model name (default: gpt-3.5-turbo-instruct).
        max_tokens (int): Maximum tokens to generate.
        temperature (float): Randomness level (0 = deterministic, 1 = creative).
        log_response (bool): Whether to log full response for debugging.

    Returns:
        str: The generated text completion, or None if error.
    """
    try:
        response = client.completions.create(
            model=model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature
        )
        if log_response:
            logger.debug(f"Full API response: {response}")

        return response.choices[0].text.strip()

    except Exception as e:
        logger.error(f"Error during completion request: {e}")
        return None


# -------------------------------------------------------------------
# Example usage
# -------------------------------------------------------------------
if __name__ == "__main__":
    test_prompt = "Write a haiku about AI and the future."
    result = get_completion(test_prompt, log_response=True)

    if result:
        print("✅ Completion result:\n", result)
    else:
        print("❌ No response received.")

## ✅ Use Case 1: Summarization
**Input:**  
Sparse vector feature indexing allows scalable search over events by turning categorical attributes into an extremely wide, mostly empty vector...

**Sample Output:**  
"It converts categorical features into sparse vectors for scalable search. The trade-off is efficiency vs. memory/compute cost at high cardinality."

In [ ]:
# ✅ Use Case 1: Summarization Code

technical_paragraph = """
Sparse vector feature indexing allows scalable search over events by turning 
categorical attributes into an extremely wide, mostly empty vector. Each distinct 
token (e.g. event_type:click, browser:mobile) maps to a coordinate set to 1 while 
all others remain 0. This removes unintended ordering between categories and keeps 
distance metrics meaningful, but at very high cardinality memory and compute 
overhead grow rapidly, motivating hashing tricks or learned embeddings.
""".strip()

summary_prompt = f"Summarize the following paragraph in 2 crisp sentences focusing on purpose and trade-offs:\n\n{technical_paragraph}\n\nSummary:"

# Call completions API using helper
summary = get_completion(
    summary_prompt, model="gpt-3.5-turbo-instruct", max_tokens=120, temperature=0.4)

print("✅ Summary:", summary)

## ✅ Use Case 2: Sentiment Classification
**Input:**  
"The user interface feels sluggish, but the reporting features are fantastic."

**Sample Output:**  
"Neutral"


In [ ]:
# ✅ Use Case 2: Sentiment Classification (Multiple Inputs)

# Sample inputs
input_texts = [
    "The user interface feels sluggish, but the reporting features are fantastic.",  # Neutral
    "I love the new dashboard design, it’s clean and very intuitive.",               # Positive
    "The system keeps crashing frequently, making it hard to use."                   # Negative
]

# Loop through each input and classify sentiment
for input_text in input_texts:
    sentiment_prompt = f"""
    Classify the sentiment of the following text strictly as Positive, Negative, or Neutral.
    If the text contains both positive and negative aspects, classify it as Neutral.

    Text: "{input_text}"

    Sentiment:
    """

    # Call completions API using helper
    sentiment = get_completion(
        sentiment_prompt,
        model="gpt-3.5-turbo-instruct",
        max_tokens=20,
        temperature=0  # deterministic output
    )

    print(f"✅ Text: {input_text}")
    print(f"✅ Sentiment: {sentiment.strip()}\n")

## ✅ Use Case 3: Multilingual Generation + Translation
**Input:**  
"Artificial Intelligence will transform education."

**Sample Output:**  
- French: "L'intelligence artificielle transformera l'éducation."  
- Spanish: "La inteligencia artificial transformará la educación."  
- Hindi: "कृत्रिम बुद्धिमत्ता शिक्षा को बदल देगी।"


In [ ]:
# ✅ Use Case 3: Multilingual Generation + Translation

input_text = "Artificial Intelligence will transform education."
languages = ["French", "Spanish", "Hindi"]

for lang in languages:
    translation_prompt = f"Translate the following text into {lang}:\n\n{input_text}"
    translation = get_completion(
        translation_prompt, max_tokens=60, temperature=0.3)
    print(f"✅ {lang}: {translation}")

## ✅ Use Case 4: Semantic Interpretation of Idiom
**Input:**  
"Kick the bucket"

**Sample Output:**  
"It means someone has died."


## ✅ Use Case 5: Factual Recall / Explanatory Response
**Input:**  
"Why is the sky blue?"

**Sample Output:**  
"The sky looks blue because sunlight is scattered by air molecules. Blue light is scattered more strongly than other colors, so we mostly see blue."

## ✅ Use Case 6: Code Generation & Explanation
**Input:**  
"Write a Python function that checks if a number is prime."

**Sample Output:**  
```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
````

**Explanation:**
"The function checks divisibility from 2 up to √n. If any divisor is found, it returns False; otherwise, True."


## ✅ Use Case 7: Conversational Agent (FAQ Assistant)

**Input:**
"What is the purpose of Git?"

**Sample Output:**
"Git is a version control system that tracks changes in code, helps collaboration, and manages project history."


## ✅ Use Case 8: Style Transfer / Tone Adaptation

**Input:**
"Please confirm receipt of this document at the earliest."

**Sample Output:**
"Hey, just let me know when you get this doc!"

## ✅ Use Case 9: Data-to-Text Generation

**Input:**
Region: North
Sales: \$50,000
Increase: 10%

**Sample Output:**
"The North region generated \$50,000 in sales, reflecting a 10% increase compared to the previous period."


## ✅ Use Case 10: Creative Writing

**Input:**
"A robot learning to paint"

**Sample Output:**
"The robot dipped its brush in bright colors, each stroke a clumsy attempt at beauty. Slowly, it learned to create sunsets and flowers. Its creators realized the machine had found its soul in art."

## ✅ Use Case 11: Question Generation

**Input:**
"The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south."

**Sample Output:**

1. What is the largest ocean on Earth?
2. How deep is the Pacific Ocean compared to other oceans?
3. Which oceans border the Pacific to the north and south?


## ✅ Use Case 12: Entity Extraction with Explanation

**Input:**
"Apple announced a new partnership with Tesla to develop AI-powered batteries in California."

**Sample Output:**

* **Apple** – Technology company forming a partnership.
* **Tesla** – Automotive/energy company collaborating on batteries.
* **California** – Location where the initiative is happening.

## ✅ Use Case 13: Paraphrasing / Rewriting

**Input:**
"The proliferation of interconnected devices has exponentially increased the complexity of ensuring cybersecurity across distributed networks."

**Sample Output:**
"With more connected devices, keeping networks secure has become much harder."



## ✅ Use Case 14: Email / Document Drafting

**Input:**
"Announce a team meeting for Friday at 3 PM to discuss the quarterly results."

**Sample Output:**
"Subject: Team Meeting – Friday at 3 PM
Hi Team,
We will have a meeting on Friday at 3 PM to review our quarterly results. Please make sure to attend.
Best regards,
\[Your Name]"

